In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime


In [2]:
wallets = pd.read_csv("wallets.csv")  # after downloading from Google Sheet


In [7]:
def get_wallet_data(wallet_address):
    return {
        "wallet_id": wallet_address,
        "total_supply": 1000,  # Replace with real API result
        "total_borrow": 600,
        "borrow_supply_ratio": 0.6
    }


In [8]:
# Example: loading wallets from Google Sheet
url = "https://docs.google.com/spreadsheets/d/1ZzaeMgNYnxvriYYpe8PE7uMEblTI0GV5GIVUnsP-sBs/export?format=csv"
wallet_df = pd.read_csv(url)


In [9]:
wallet_features = []

for wallet in wallet_df['wallet_id']:
    data = get_wallet_data(wallet)
    wallet_features.append(data)

features_df = pd.DataFrame(wallet_features)


In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_to_normalize = ['total_supply', 'total_borrow', 'borrow_supply_ratio']
features_df[features_to_normalize] = scaler.fit_transform(features_df[features_to_normalize])


In [11]:
# Example weights
features_df['score_raw'] = (
    0.4 * features_df['borrow_supply_ratio'] +
    0.3 * features_df['total_borrow'] +
    0.3 * features_df['total_supply']
)

# Scale to 0–1000
features_df['score'] = (
    (features_df['score_raw'] - features_df['score_raw'].min()) /
    (features_df['score_raw'].max() - features_df['score_raw'].min())
) * 1000


In [12]:
features_df[['wallet_id', 'score']].to_csv("wallet_scores.csv", index=False)


## Data Collection

- Wallets were loaded from the provided Google Sheet.
- For each wallet, mock data was generated using a function.
- In production, this would connect to Compound's subgraph or REST API.

## Feature Selection

- **total_supply**: Total value supplied to the protocol
- **total_borrow**: Total borrowed amount
- **borrow_supply_ratio**: Indicator of leverage risk

## Risk Scoring Logic

The score is calculated using a weighted sum:

- 40%: borrow/supply ratio (higher ratio = more risk)
- 30%: total borrow (larger borrow = more exposure)
- 30%: total supply (more supply = safer profile)

All scores are normalized to a 0–1000 scale.


In [16]:
features_df[['wallet_id', 'score']].to_csv("wallet_scores.csv", index=False)
